# Run Hugging Face `facebook/opt-66b` autoregressive sampling on Inf2

In this example we compile and deploy the Hugging Face [facebook/opt-66b](https://huggingface.co/facebook/opt-66b) model for tensor parallel inference on Neuron using the `transformers-neuronx` package.

The example has the following main sections:
1. Set up the Jupyter Notebook
1. Install dependencies
1. Download and construct the model
1. Split the model `state_dict` into multiple files
1. Perform autoregressive sampling using tensor parallelism

This Jupyter Notebook should be run on an Inf2 instance (`inf2.48xlarge`).

## Set up the Jupyter Notebook

The following steps set up Jupyter Notebook and launch this tutorial:
1. Clone the [AWS Neuron Samples](https://github.com/aws-neuron/aws-neuron-samples) repo to your instance using
```
git clone https://github.com/aws-neuron/aws-neuron-samples.git
```
2. Navigate to the `transformers-neuronx` inference samples folder
```
cd aws-neuron-samples/torch-neuronx/transformers-neuronx/inference
```
3. Follow the instructions in [Jupyter Notebook QuickStart](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/notebook/setup-jupyter-notebook-steps-troubleshooting.html) to run Jupyter Notebook on your instance.
4. Locate this tutorial in your Jupyter Notebook session (`facebook-opt-66b-sampling.ipynb`) and launch it. Follow the rest of the instructions in this tutorial. 

## Install Dependencies

This tutorial requires the following pip packages:

 - `torch-neuronx`
 - `neuronx-cc`
 - `transformers`
 - `transformers-neuronx`

Most of these packages will be installed when configuring your environment using the [torch-neuronx inference setup guide](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/frameworks/torch/torch-neuronx/setup/setup-inference.html). The additional dependencies must be installed here:

In [ ]:
!pip install transformers-neuronx -U

## Download and construct the model

We download and construct the `facebook/opt-66b` model using the Hugging Face
`from_pretrained` method.

Note that downloading the model checkpoint from the Hugging Face repo will take a few minutes.

In [ ]:
from transformers.models.opt import OPTForCausalLM

hf_model = OPTForCausalLM.from_pretrained('facebook/opt-66b', low_cpu_mem_usage=True)

## Split the model state_dict into multiple files

For the sake of reducing host memory usage, it is recommended to save the model `state_dict` as
multiple files, as opposed to one monolithic file given by `torch.save`. This "split-format"
`state_dict` can be created using the `save_pretrained_split` function. With this checkpoint format,
the Neuron model loader can load parameters to the Neuron device high-bandwidth memory (HBM) directly
by keeping at most one layer of model parameters in the CPU main memory.

To reduce memory usage during compilation and deployment, we cast the attention and mlp to `float16` precision before saving them. We keep the layernorms in `float32`. To do this, we implement a callback function that casts each layer in the model.

Note that splitting the model weights will take a few minutes.

In [ ]:
import torch
from transformers_neuronx.module import save_pretrained_split

def amp_callback(model, dtype):
    # cast attention and mlp to low precision only; layernorms stay as f32
    for block in model.model.decoder.layers:
        block.self_attn.to(dtype)
        block.fc1.to(dtype)
        block.fc2.to(dtype)
    model.lm_head.to(dtype)

amp_callback(hf_model, torch.float16)
save_pretrained_split(hf_model, './opt-66b-split')

## Perform autoregressive sampling using tensor parallelism

Now we have all the necessary files for running `facebook/opt-66b` autoregressive sampling. 

To get a large language model working on Inf2 & Trn1, tensor parallelism is used to split weights and data across multiple NeuronCores. Each NeuronCore has 16GB of memory. As a rule of thumb, the total space required per NeuronCore will be at least `2 * number of model parameters` for a `float16` casted model. In reality, the total space required is often greater due to the key value cache, which grows with sequence lenght. This memory usage determines the minimum viable instance size since the amount of memory that will be allocated on one NeuronCore is directly proportional to the parallelism degree (`tp_degree`), or rather the number of physical NeuronCores per instance. The parallelism degree must be chosen to ensure that the memory usage per NeuronCore will be less than the physical 16GB limit. While this determines the minimum instance sizing, further decreasing the memory usage per NeuronCore by using a larger instance and a higher `tp_degree` should result in a faster model

We will use the Neuron `OPTForSampling` class to implement tensor parallelism. The default model config supports sampling up to sequence length 2048, and we set batch size to 2. Tensor-parallelism is enabled through the 
`tp_degree=24` argument. Internally, the Neuron tensor manipulator will shard and duplicate tensors to multiple
NeuronCores (24 in this case) to enable tensor-parallel computations on multiple NeuronCores. The model computational graph is compiled by `neuronx-cc` for optimized inference on Neuron.

Note that compiling the model will take a long time. However, compilation is one-time process so it will not impact deployment performance. 

In [ ]:
import time
import torch
from transformers import AutoTokenizer
from transformers_neuronx.opt.model import OPTForSampling

# load facebook/opt-66b to NeuronCores with 24-way tensor parallel
# enable float16 casting
neuron_model = OPTForSampling.from_pretrained('./opt-66b-split', batch_size=2, tp_degree=24, amp='f16')
neuron_model.to_neuron()

# construct a tokenizer and encode prompt text
tokenizer = AutoTokenizer.from_pretrained('facebook/opt-66b')
batch_prompts = [
    "Hello, I'm a language model,",
    "Welcome to Amazon Elastic Compute Cloud,",
]
input_ids = torch.as_tensor([tokenizer.encode(text) for text in batch_prompts])

with torch.inference_mode():
    start = time.time()
    generated_sequences = neuron_model.sample(input_ids, sequence_length=2048)
    elapsed = time.time() - start

generated_sequences = [tokenizer.decode(seq) for seq in generated_sequences]
print(f'generated sequences {generated_sequences} in {elapsed} seconds')

The `facebook/opt-66b` model is not supported on `trn1.32xlarge`. The `facebook/opt-66b` model number of attention heads is 72, so the largest tensor parallel degree that is supported on Trn1 and is divisor of 72 is 8. However, the total size of the `facebook/opt-66b` model and its key-value caches is greater than 8 * 16 GB in `float16`, so it will not fit into memory using this tensor parallel degree. Thus, `inf2.48xlarge` with tensor parallel degree = 24 is the supported configuration for the `facebook/opt-66b` model.